In [18]:
#!pip install langchain
#!pip install -U langchain-community
#!pip install assemblyai

In [1]:
from langchain.document_loaders import AssemblyAIAudioTranscriptLoader
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [27]:
# Configurar la API Key
import os

#API KEY DE Assembly AI
#api_key = "bf0a93ab806b4c45a3b7f7c22ba03837"

# Obtener la API Key desde la variable de entorno
api_key2 = os.getenv("ASSEMBLYAI_API_KEY")

#file_audio = "prueba_voz_LLM.mp3"
file_audio = "voz_llm_2.mp3"

loader_audio = AssemblyAIAudioTranscriptLoader(file_path= file_audio, api_key=api_key2)
documentos = loader_audio.load()

print(documentos[0].page_content)
#for i in documentos:
#    print(i)

Brian itraho, commissioner, intrigue certificate.


In [28]:
print(documentos[0].page_content)

Brian itraho, commissioner, intrigue certificate.


In [29]:
from langchain_community.llms import Ollama  # Cambiar OpenAI por Ollama

#llm_ollama = Ollama(model="mistral", temperature=0)
llm_ollama2 = Ollama(model="tinyllama", temperature=0)

In [30]:
cadena_voz = load_qa_chain(llm_ollama2, chain_type="stuff")

respuesta = cadena_voz.run(input_documents = documentos, question="Quien es y de que trabaja?")

print(respuesta)

Question: Who is and what does he/she do?

Answer: Brían Itrahó, commissioner, intrigue certificate.


In [36]:
#!pip install gTTS

In [37]:
from gtts import gTTS
# Convertir la respuesta a voz (TTS)
tts = gTTS(text=respuesta, lang="es")
tts.save("respuesta.mp3")
os.system("mpg321 respuesta.mp3")  # Reproducir el archivo de audio

1

In [40]:
#!pip install SpeechRecognition
#!pip install pydub

### Usando Speech Reconocimint para pasar de VOZ a Texto

In [2]:
import speech_recognition as sr
from pydub import AudioSegment

In [ ]:
# Función para convertir MP3 a WAV
#def convert_mp3_to_wav(mp3_file, wav_file):
#    audio = AudioSegment.from_mp3(mp3_file)
#    audio.export(wav_file, format="wav")

# Función para transcribir el archivo de audio
def transcribir_audio(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio, language="es-ES")
            print("Texto transcrito:", text)
            return text
        except sr.UnknownValueError:
            print("No se pudo entender el audio")
        except sr.RequestError as e:
            print(f"Error al solicitar resultados: {e}")

In [6]:
# Archivo de audio
#mp3_file = "voz_llm_2.mp3"
wav_file = "voz_llm_2.wav"

# Convertir MP3 a WAV (si es necesario)
#convert_mp3_to_wav(mp3_file, wav_file)

# Transcribir el archivo WAV
texto_transcrito = transcribir_audio(wav_file)

Texto transcrito: Hola Mi nombre es Brian y trabajo como ingeniero de Inteligencia artificial


In [7]:
texto_transcrito

'Hola Mi nombre es Brian y trabajo como ingeniero de Inteligencia artificial'

In [ ]:
import speech_recognition as sr  #procesar voz y capturar lo que el usuario dice a través del micrófono.
from langchain_community.llms import Ollama # Modelo LLM
from gtts import gTTS  # Para convertir respuesta de LLM en voz
import os  # Acceder a archivos del sistema

# Función para capturar audio en tiempo real
def capturar_audio():
    # Objeto Recognizer para manejar la captura y transcripción de audio
    recognizer = sr.Recognizer()
    with sr.Microphone() as source: #Abre el micrófono como fuente de audio
        print("Escuchando...")
        audio = recognizer.listen(source) #Capturamos el audio del micrófono.
        try:
            #Transcribir el audio a texto en español
            text = recognizer.recognize_google(audio, language="es-ES")
            print("Usuario:", text)
            return text  #texto transcrito
        except sr.UnknownValueError:
            print("No se pudo entender el audio")
        except sr.RequestError as e:
            print(f"Error al solicitar resultados: {e}")

# Función para generar una respuesta con el modelo de lenguaje
def generar_respuesta(texto_usuario):
    # Cargar el modelo de Ollama
    llm_ollama = Ollama(model="tinyllama", temperature=0.3)

    # Generar una respuesta basada en lo que dijo el usuario
    respuesta = llm_ollama(texto_usuario)
    return respuesta

# Función para convertir texto a voz
def texto_a_voz(texto, archivo_salida="respuesta.mp3"):
    tts = gTTS(text=texto, lang="es") #objeto para convertir el texto en voz en español
    tts.save(archivo_salida) #Guarda la voz generada en un archivo MP3
    print(f"Agente:", texto)
    os.system(f"mpg321 {archivo_salida}")  # Reproducir el archivo de audio

# Interacción en tiempo real
print("Bienvenido al Agente de Voz de ATOM. ¿En qué puedo ayudarte hoy?")
texto_a_voz("Bienvenido al Agente de Voz de ATOM. ¿En qué puedo ayudarte hoy?")

while True: #Creamos un bucle infinito para mantener la conversación.
    # 1. Captura lo que el usuario dice y lo transcribe a texto
    texto_usuario = capturar_audio()
    if texto_usuario:
        # 2. LLM genera una respuesta basada en lo que dijo el usuario
        respuesta = generar_respuesta(texto_usuario)
        
        # 3. Se Convierte la respuesta de texto a voz y reproducirla
        texto_a_voz(respuesta)

In [ ]:
##

In [4]:
import json
def parse_respuesta(respuesta):
    """Parsea la respuesta del modelo LLM asegurando que sea un JSON válido."""
    try:
        if isinstance(respuesta, str):  # Si la respuesta es un string, intenta convertirla en JSON
            respuesta = json.loads(respuesta.strip())
        if isinstance(respuesta, dict):  # Si ya es un diccionario, verificar que tenga la estructura esperada
            return {
                "intencion": respuesta.get("intencion", ""),
                "nombre": respuesta.get("nombre"),
                "empresa": respuesta.get("empresa"),
                "necesidad": respuesta.get("necesidad"),
                "presupuesto": respuesta.get("presupuesto"),
                "respuesta": respuesta.get("respuesta", "No tengo información para responder.")
            }
        else:
            raise ValueError("La respuesta no es un diccionario válido")
    except json.JSONDecodeError:
        print("Error al convertir la respuesta en JSON:", respuesta)
        return {"respuesta": respuesta}  # Devolver un valor por defecto en caso de error